### **使用 Keras 實現資料增強**

在 Keras 中，可以使用 `ImageDataGenerator` 來實現常見的資料增強技術。這個類別可以對訓練數據進行實時的增強處理。

#### **Keras 資料增強實作**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10

# 下載並預處理 CIFAR-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 正規化數據，使像素值在 0 到 1 之間
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 定義資料增強技術
datagen = ImageDataGenerator(
    rotation_range=30,         # 隨機旋轉 30 度內
    width_shift_range=0.1,     # 水平平移 10% 圖像寬度
    height_shift_range=0.1,    # 垂直平移 10% 圖像高度
    horizontal_flip=True,      # 隨機水平翻轉
    zoom_range=0.2,            # 隨機縮放 20%
    shear_range=0.2            # 隨機錯切變換
)

# 使用訓練數據來計算所需的任何統計數據（例如均值和方差），以便進行資料增強
datagen.fit(x_train)

# 構建 CNN 模型（示例模型）
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),  # 第一個卷積層，32 個 3x3 的濾波器，ReLU 激活函數
    tf.keras.layers.MaxPooling2D((2, 2)),  # 最大池化層，池化窗口大小為 2x2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),  # 第二個卷積層，64 個 3x3 的濾波器，ReLU 激活函數
    tf.keras.layers.MaxPooling2D((2, 2)),  # 最大池化層，池化窗口大小為 2x2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),  # 第三個卷積層，64 個 3x3 的濾波器，ReLU 激活函數
    tf.keras.layers.MaxPooling2D((2, 2)),  # 最大池化層，池化窗口大小為 2x2
    tf.keras.layers.Flatten(),  # 將多維輸出展平成一維
    tf.keras.layers.Dense(64, activation='relu'),  # 全連接層，64 個神經元，ReLU 激活函數
    tf.keras.layers.Dense(10, activation='softmax')  # 輸出層，10 個神經元（對應 10 個類別），softmax 激活函數
])

# 編譯模型，指定優化器、損失函數和評估指標
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 使用資料增強進行訓練
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),  # 使用資料增強生成器
                    steps_per_epoch=len(x_train) // 64,  # 每個 epoch 的步數
                    epochs=10,  # 訓練 10 個 epoch
                    validation_data=(x_test, y_test))  # 驗證數據


### **使用 PyTorch 實現資料增強**

在 PyTorch 中，資料增強可以通過 `torchvision.transforms` 來實現，這是一個方便的數據增強模組。

#### **PyTorch 資料增強實作**

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# 定義資料增強技術
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),     # 隨機水平翻轉
    transforms.RandomRotation(30),         # 隨機旋轉 30 度內
    transforms.RandomResizedCrop(32),      # 隨機裁剪後調整到 32x32 大小
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # 色彩調整
    transforms.ToTensor(),                 # 轉換為張量
])

# 下載 CIFAR-10 資料集並應用資料增強
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 定義模型（簡單的 CNN 模型）
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 第一個卷積層，32 個 3x3 的濾波器，padding=1 保持輸入大小不變
    torch.nn.ReLU(),  # ReLU 激活函數
    torch.nn.MaxPool2d(kernel_size=2),  # 最大池化層，池化窗口大小為 2x2
    torch.nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 第二個卷積層，64 個 3x3 的濾波器，padding=1 保持輸入大小不變
    torch.nn.ReLU(),  # ReLU 激活函數
    torch.nn.MaxPool2d(kernel_size=2),  # 最大池化層，池化窗口大小為 2x2
    torch.nn.Flatten(),  # 將多維輸出展平成一維
    torch.nn.Linear(64 * 8 * 8, 128),  # 全連接層，輸入大小為 64*8*8，輸出大小為 128
    torch.nn.ReLU(),  # ReLU 激活函數
    torch.nn.Linear(128, 10)  # 輸出層，10 個神經元（對應 10 個類別）
)

# 訓練模型（示例）
criterion = torch.nn.CrossEntropyLoss()  # 定義損失函數為交叉熵損失
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 定義優化器為 Adam，學習率為 0.001

for epoch in range(10):  # 訓練 10 個 epoch
    running_loss = 0.0
    for images, labels in train_loader:  # 遍歷訓練數據
        optimizer.zero_grad()  # 清空梯度
        outputs = model(images)  # 前向傳播
        loss = criterion(outputs, labels)  # 計算損失
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新參數
        running_loss += loss.item()  # 累加損失
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')  # 輸出每個 epoch 的平均損失
    